In [2]:
import numpy as np
import tensorflow as tf
import time
from IPython.display import display, JSON
import pandas as pd

In [12]:
# 1. 데이터 생성 함수
def generate_continuous_data(prev_data=None):
    if prev_data is None:  # 초기값 설정
        return {
            'LONGITUDE': np.random.uniform(126.0, 129.0),  # 대한민국 경도 범위
            'LATITUDE': np.random.uniform(34.0, 38.0),    # 대한민국 위도 범위
            'HEADING': np.random.uniform(0, 360),         # 0 ~ 360도
            'SPEED': np.random.uniform(0, 100),           # 0 ~ 100 km/h
            'BRAKE_STATUS': 0,                            # 초기 브레이크 상태
            'ACC_SEC': 0,                                 # 초기 가속도
            'CURRENT_LANE': np.random.randint(1, 4)       # 1 ~ 3차선
        }

    # 연속적 변화 생성
    return {
        'LONGITUDE': prev_data['LONGITUDE'] + np.random.uniform(-0.1, 0.1),  # 약간의 이동
        'LATITUDE': prev_data['LATITUDE'] + np.random.uniform(-0.1, 0.1),    # 약간의 이동
        'HEADING': (prev_data['HEADING'] + np.random.uniform(-5, 5)) % 360,        # 방향의 변화
        'SPEED': max(0, min(prev_data['SPEED'] + np.random.uniform(-2, 2), 120)),  # 속도 변화 (0~120 km/h)
        'BRAKE_STATUS': np.random.uniform(60, 80) if np.random.rand() < 0.5 else max(0, prev_data['BRAKE_STATUS'] - np.random.uniform(0, 10)),
        'ACC_SEC': np.random.uniform(-10, 10),  # 가속도 변화
        'CURRENT_LANE': prev_data['CURRENT_LANE'] + np.random.choice([-1, 0, 1], p=[0.1, 0.8, 0.1])  # 차선 이동
    }

In [3]:
# 2. 저장된 모델 로드
def load_model(model_path):
    return tf.keras.models.load_model(model_path)

In [4]:
# 리스트를 데이터프레임으로 출력하는 함수
def display_data(buffer_a, buffer_b):
    df_a = pd.DataFrame(buffer_a, columns=['LONGITUDE', 'LATITUDE', 'HEADING', 'SPEED', 'BRAKE_STATUS', 'ACC_SEC', 'CURRENT_LANE'])
    df_b = pd.DataFrame(buffer_b, columns=['LONGITUDE', 'LATITUDE', 'HEADING', 'SPEED', 'BRAKE_STATUS', 'ACC_SEC', 'CURRENT_LANE'])

    print("=== A Vehicle Data ===")
    display(df_a)
    print("=== B Vehicle Data ===")
    display(df_b)

In [5]:
# 3. 10초 단위로 데이터 묶기 및 예측
def simulate(model):
    print("Simulating real-time data...\n")
    # 버퍼 초기화
    buffer_a = []  # A 차량의 10초 단위 데이터를 저장할 버퍼
    buffer_b = []  # B 차량의 10초 단위 데이터를 저장할 버퍼

    # 초기 데이터 생성
    prev_data_a = generate_continuous_data()
    prev_data_b = generate_continuous_data()

    while True:
        # 연속적 데이터 생성
        data_a = generate_continuous_data(prev_data_a)
        data_b = generate_continuous_data(prev_data_b)

        # 이전 데이터를 업데이트
        prev_data_a = data_a
        prev_data_b = data_b

        buffer_a.append(list(data_a.values()))  # A 데이터를 버퍼에 추가
        buffer_b.append(list(data_b.values()))  # B 데이터를 버퍼에 추가
        print(f"A data: {data_a}")
        print(f"B data: {data_b}")
        print('================================')
        
        # 10초마다 예측
        if len(buffer_a) == 10 and len(buffer_b) == 10:
            print("\n=== 10-Second Data Batch ===")
            display_data(buffer_a, buffer_b)
            
            # 모델 입력 (LSTM 모델은 시계열 데이터로 처리)
            input_data_a = np.expand_dims(np.array(buffer_a), axis=0)  # (1, 10, 7) 형태
            
            # A 모델 예측 수행
            prediction_a = model.predict(input_data_a)
            print(f"\nA Prediction: {prediction_a}")
            
            # 결과 반환 (A에서 돌발 상황 감지 시)
            if prediction_a[0][0] > 0.5:  # 예: 위험 확률 > 50%
                print("⚠️ 경고 : A 차량에서 돌발 상황 감지!\n")
                return {
                    "status": "Hazard",
                    "a_latitude": buffer_a[-1][1],  # A의 마지막 데이터의 위도
                    "a_longitude": buffer_a[-1][0],  # A의 마지막 데이터의 경도
                    "b_latitude": buffer_b[-1][1],  # B의 마지막 데이터의 위도
                    "b_longitude": buffer_b[-1][0]  # B의 마지막 데이터의 경도
                }
            
            # 버퍼 초기화
            buffer_a = []
            buffer_b = []
        
        # 1초 대기
        time.sleep(1)

In [6]:
# 4. 거리와 위치 계산 함수
def calculate_relative_position(lat_a, lon_a, lat_b, lon_b):
    try:
        # 거리 계산
        R = 6371e3  # 지구반지름
        phi1, phi2 = np.radians(lat_a), np.radians(lat_b)
        delta_phi = np.radians(lat_b - lat_a)
        delta_lambda = np.radians(lon_b - lon_a)
        a = np.sin(delta_phi / 2) ** 2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2) ** 2
        c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
        distance = R * c

        # 방향 계산
        x = np.sin(delta_lambda) * np.cos(phi2)
        y = np.cos(phi1) * np.sin(phi2) - np.sin(phi1) * np.cos(phi2) * np.cos(delta_lambda)
        bearing = (np.degrees(np.arctan2(x, y)) + 360) % 360

        # 위치 관계 구분
        if distance < 100:  # 100m 이내에서만 상대 위치 구분
            if 0 <= bearing < 45 or 315 <= bearing <= 360:
                return "앞 차량", distance
            elif 135 <= bearing < 225:
                return "뒤 차량", distance
            elif 45 <= bearing < 135:
                return "오른쪽 차량", distance
            elif 225 <= bearing < 315:
                return "왼쪽 차량", distance

        return "멀리 떨어진 차량", distance

    except Exception as e:
        print(f"❌ Error in calculate_relative_position: {e}")
        return "잘못된 계산", float('inf')

In [13]:
if __name__ == "__main__":
    model_path = "v2x_model.h5"
    model = load_model(model_path)
    
    # 돌발 상황 감지까지 시뮬레이션 반복
    hazard_data = simulate(model)
    
    # 상대 위치 계산
    position, distance = calculate_relative_position(
        hazard_data["a_latitude"], hazard_data["a_longitude"],
        hazard_data["b_latitude"], hazard_data["b_longitude"]
    )

    # 상대 위치에 따른 처리
    print(f"⚠️ B 차량의 {position}인 A에서 돌발 상황이 감지되었습니다. 주의하여 운전하세요! 거리 : {distance} 미터\n")

Simulating real-time data...

A data: {'LONGITUDE': 128.63847243509505, 'LATITUDE': 37.67279265101839, 'HEADING': 337.7256348375435, 'SPEED': 29.83425555937372, 'BRAKE_STATUS': 62.60745151964082, 'ACC_SEC': -5.6982558303408055, 'CURRENT_LANE': 3}
B data: {'LONGITUDE': 126.40619350823624, 'LATITUDE': 37.20404817538775, 'HEADING': 265.989640294372, 'SPEED': 0.6743331819680796, 'BRAKE_STATUS': 79.95947534493104, 'ACC_SEC': 7.900356867821834, 'CURRENT_LANE': 2}
A data: {'LONGITUDE': 128.73449987231305, 'LATITUDE': 37.66540060700434, 'HEADING': 337.7477645607946, 'SPEED': 30.79404303444582, 'BRAKE_STATUS': 57.55479546274188, 'ACC_SEC': -7.212850477828274, 'CURRENT_LANE': 3}
B data: {'LONGITUDE': 126.36853041186201, 'LATITUDE': 37.21153239753431, 'HEADING': 270.76363908550945, 'SPEED': 0, 'BRAKE_STATUS': 64.70565238704279, 'ACC_SEC': -7.306768765799352, 'CURRENT_LANE': 2}
A data: {'LONGITUDE': 128.70227062891777, 'LATITUDE': 37.65341597982285, 'HEADING': 338.36068635532064, 'SPEED': 32.32111

,LONGITUDE,LATITUDE,HEADING,SPEED,BRAKE_STATUS,ACC_SEC,CURRENT_LANE
0,128.638472,37.672793,337.725635,29.834256,62.607452,-5.698256,3
1,128.734500,37.665401,337.747765,30.794043,57.554795,-7.212850,3
2,128.702271,37.653416,338.360686,32.321116,51.368465,-8.253481,2
3,128.720746,37.648596,342.995485,32.679783,48.018794,2.881353,2
4,128.746206,37.598957,340.544656,31.200867,46.685438,1.974342,2
5,128.779561,37.542349,339.041870,31.287682,44.041197,8.149796,2
6,128.859089,37.549389,337.795322,30.593011,41.690429,-5.210935,2
7,128.930475,37.511071,342.714332,29.223974,70.878908,4.488347,3
8,128.840490,37.607628,345.180075,29.142404,68.282469,5.967052,3
9,128.881313,37.532191,343.018641,28.431134,65.422318,-8.893550,3


=== B Vehicle Data ===


,LONGITUDE,LATITUDE,HEADING,SPEED,BRAKE_STATUS,ACC_SEC,CURRENT_LANE
0,126.406194,37.204048,265.989640,0.674333,79.959475,7.900357,2
1,126.368530,37.211532,270.763639,0.000000,64.705652,-7.306769,2
2,126.385131,37.210249,266.098695,1.206420,74.431565,0.668069,2
3,126.386926,37.228957,268.839660,2.246500,73.541738,7.953067,2
4,126.349829,37.252010,268.751355,0.847042,70.273625,-2.236379,2
5,126.261769,37.318965,273.223356,0.893493,63.113706,-5.696180,2
6,126.261590,37.259355,276.474135,2.409397,61.800955,7.156040,2
7,126.317048,37.278998,278.048911,3.736776,57.083567,-0.988805,2
8,126.313064,37.211808,282.526100,5.322277,73.417641,-0.479764,2
9,126.243658,37.235766,283.251445,6.407291,73.298703,7.013207,2


1/1 [==============================] - 0s 246ms/step

A Prediction: [[0.00034208]]
A data: {'LONGITUDE': 128.8161384444895, 'LATITUDE': 37.58300415476434, 'HEADING': 346.75384404703544, 'SPEED': 29.47443241657577, 'BRAKE_STATUS': 58.417484014090434, 'ACC_SEC': 3.3859605498478977, 'CURRENT_LANE': 3}
B data: {'LONGITUDE': 126.25840963015469, 'LATITUDE': 37.23444410230884, 'HEADING': 280.5951302132478, 'SPEED': 6.949753837025775, 'BRAKE_STATUS': 69.06474067142953, 'ACC_SEC': 0.015381724227536253, 'CURRENT_LANE': 3}
A data: {'LONGITUDE': 128.75007318909377, 'LATITUDE': 37.56515203572363, 'HEADING': 349.22137522892825, 'SPEED': 28.65371656616091, 'BRAKE_STATUS': 57.83336465396687, 'ACC_SEC': -4.9867122022302945, 'CURRENT_LANE': 3}
B data: {'LONGITUDE': 126.25710557385446, 'LATITUDE': 37.23610771645568, 'HEADING': 285.5416892661849, 'SPEED': 5.01830060407551, 'BRAKE_STATUS': 60.7557704568896, 'ACC_SEC': -5.5602414582434285, 'CURRENT_LANE': 3}
A data: {'LONGITUDE': 128.68724446159823, 'LATITU

...

,LONGITUDE,LATITUDE,HEADING,SPEED,BRAKE_STATUS,ACC_SEC,CURRENT_LANE
0,128.735311,37.298014,350.779969,32.510837,66.289687,5.197686,2
1,128.744176,37.310323,351.977238,33.523327,62.256324,-2.674912,2
2,128.687621,37.310694,350.791296,33.325506,77.303624,0.624295,2
3,128.698630,37.264793,347.104756,33.459497,60.034217,-5.700640,2
4,128.636002,37.204776,351.954003,34.572682,67.812803,1.261722,2
5,128.560516,37.294910,354.129791,34.721300,62.448461,8.576955,2
6,128.615903,37.366128,358.458455,36.129769,55.756581,9.464467,3
7,128.552475,37.275534,354.715170,37.291623,48.634197,4.952284,3
8,128.550627,37.207710,355.896414,36.533614,66.572443,-2.874762,3
9,128.593831,37.168277,351.871008,36.371959,79.473922,-5.114204,3


=== B Vehicle Data ===


,LONGITUDE,LATITUDE,HEADING,SPEED,BRAKE_STATUS,ACC_SEC,CURRENT_LANE
0,126.188235,37.356340,274.615624,9.781763,71.527341,4.427599,5
1,126.126571,37.297759,279.570996,10.983155,69.662485,-7.499082,6
2,126.081059,37.240059,284.155855,9.967092,74.226332,-7.614388,6
3,126.086726,37.149300,281.614207,8.640288,65.706883,0.945215,6
4,126.040939,37.173962,283.167874,8.206338,63.745287,-8.570138,6
5,125.950336,37.093861,283.279374,6.685458,53.831692,-7.066316,6
6,125.851127,37.054552,278.875166,6.264422,45.152066,5.955055,6
7,125.757543,37.007154,282.614606,7.982761,77.750520,-8.705253,7
8,125.659917,37.043213,278.915736,7.233078,69.790766,9.642714,7
9,125.746139,36.982119,278.149758,7.147836,78.120200,-6.268480,7


1/1 [==============================] - 0s 4ms/step

A Prediction: [[0.00025828]]
A data: {'LONGITUDE': 128.59353306629964, 'LATITUDE': 37.26689637757812, 'HEADING': 356.2221990267385, 'SPEED': 34.377875653803045, 'BRAKE_STATUS': 72.29054251685106, 'ACC_SEC': 6.39108617339685, 'CURRENT_LANE': 3}
B data: {'LONGITUDE': 125.76559985090428, 'LATITUDE': 36.930454162156046, 'HEADING': 282.64121857874267, 'SPEED': 6.152044940484897, 'BRAKE_STATUS': 72.90913559813153, 'ACC_SEC': -8.091255425452431, 'CURRENT_LANE': 7}
A data: {'LONGITUDE': 128.5078659617531, 'LATITUDE': 37.244609367352304, 'HEADING': 358.8109514041273, 'SPEED': 32.70687294908563, 'BRAKE_STATUS': 77.06724926690384, 'ACC_SEC': 3.0605498960527378, 'CURRENT_LANE': 3}
B data: {'LONGITUDE': 125.74341477407289, 'LATITUDE': 36.917544560111, 'HEADING': 277.8773280324433, 'SPEED': 6.752816364535354, 'BRAKE_STATUS': 67.12951219756584, 'ACC_SEC': 3.7886580735154247, 'CURRENT_LANE': 6}
A data: {'LONGITUDE': 128.504449360151, 'LATITUDE': 37.2

KeyboardInterrupt: 